In [1]:
from DataDownload import *
from PreProcessing import *
from EventIdentification import *
from CataloguedEvents import *
from PostProcessing import *
import matplotlib.pyplot as plt
from obspy import UTCDateTime
import torch
import sys
from ReportGeneration import *

Initiating Parameters for Report Content Generation

In [2]:
# Set date and station for studying423
date = UTCDateTime("2024-04-23") 
station = ['AM','R50D6','https://data.raspberryshake.org']

# Generate a path for all files to be saved to
date_folder = generate_data_path(date, station)
content_path = os.path.join(date_folder, "generated")
os.makedirs(content_path, exist_ok=True)

Reading Saved Files

In [3]:
saved_catalogue, provider = read_catalogue_file(date_folder)
saved_df_completed = read_csv_from_path(date_folder, date, station, 'completed')
saved_processed_stream = read_stream_from_path(date_folder, date, station, 'processed')
saved_annotated_stream = read_stream_from_path(date_folder, date, station, 'processed.annotated')

Loaded data from D:\Dissertation\data\AM.R50D6\2024-04-23\2024-04-23_IRIS.catalogue.xml
Loaded data from D:\Dissertation\data\AM.R50D6\2024-04-23\2024-04-23.completed.csv
Loaded stream from D:\Dissertation\data\AM.R50D6\2024-04-23\2024-04-23_AM.R50D6..Z.processed.mseed
Loaded stream from D:\Dissertation\data\AM.R50D6\2024-04-23\2024-04-23_AM.R50D6..Z.processed.annotated.mseed


Save Catalogue Globe Plot To File

In [4]:
plot_catalogue(saved_catalogue,station, date,  fill_map = True, path=content_path,show=True, save=True);

Saved plot to D:\Dissertation\data\AM.R50D6\2024-04-23\generated\catalogued_plot_2024-04-23.png


<Figure size 640x480 with 0 Axes>

Save Wave+Annotation Plots For Matched Events To File Using Event ID

In [5]:
matched_events = saved_df_completed[(saved_df_completed['catalogued'] == True) & (saved_df_completed['detected'] == True)]

for _, row in matched_events.iterrows():
    earthquake_info = get_event_info(row)  
    
    if matched_events.empty:
        continue
    plot_predictions_wave(saved_processed_stream,saved_annotated_stream, earthquake_info, path=content_path,show=False, save=True)

Saved plot to D:\Dissertation\data\AM.R50D6\2024-04-23\generated\annotation_11832031.png
Saved plot to D:\Dissertation\data\AM.R50D6\2024-04-23\generated\annotation_11831877.png


Statistics For Matched Events

In [6]:
for _, row in matched_events.iterrows():
    earthquake_info = get_event_info(row)  
    
    if matched_events.empty:
        continue
    print_event_statistics(earthquake_info)

Event ID: smi:service.iris.edu/fdsnws/event/1/query?eventid=11832031
Earthquake Time: 2024-04-23T13:05:39.336000
Location: Lat 50.0999, Long -28.9704
Magnitude: 5.0 mww
Depth: 9121.0 m
Distance to Station: 1921.27 km

Predicted P time: 2024-04-23T13:09:42.848903
Detected P time: 2024-04-23T13:09:48.950000
Predicted S time: 2024-04-23T13:13:21.000711
Detected S time: nan
Prediction confidence: P: 0.7034019827842712   S: None
----------------------------------------
Event ID: smi:service.iris.edu/fdsnws/event/1/query?eventid=11831877
Earthquake Time: 2024-04-23T02:20:59.691000
Location: Lat 23.8259, Long 121.6333
Magnitude: 5.2 Mwr
Depth: 10000.0 m
Distance to Station: 9764.02 km

Predicted P time: 2024-04-23T02:33:49.058574
Detected P time: 2024-04-23T02:33:40.040000
Predicted S time: 2024-04-23T02:44:32.124338
Detected S time: nan
Prediction confidence: P: 0.8594319820404053   S: None
----------------------------------------


In [13]:
def create_earthquake_report_html(matched_events, base_image_path, date, station, catalogue_provider):
    network, station_code, _ = station
    date_str = date.strftime('%Y-%m-%d')
    output_html_file = os.path.join(base_image_path, f"{date_str}_report.html")
    catalog_image_filename = f"catalogued_plot_{date_str}.png"  # Use filename instead of path

    html_content = f"""
    <html>
    <head>
        <title>Earthquake Report for {date_str}</title>
        <style>
            body {{ font-family: Arial, sans-serif; }}
            .event {{ margin-bottom: 40px; }}
            .event img {{ width: 100%; max-width: 600px; }}
            .catalogue_plot img {{ width: 100%; max-width: 650px; }}
            .statistics {{ margin-top: 5px; }}
        </style>
    </head>
    <body>
        <h1>Earthquake Report for {date_str}</h1>
        <p><strong>Station:</strong> {network}.{station_code}</p>
        <p><strong>Catalogue Provider:</strong> {catalogue_provider}</p>
        </br>
        </br>
        
        <div class="catalogue_plot">
            <p><strong>Catalogued Events:</strong></p>
            <img src="{catalog_image_filename}" alt="Catalogue Overview Map">  <!-- Use filename here -->
        </div>
        </br>
        </br>
        <p><strong>Detected Events From Catalogue:</strong></p>
    """
    
    for _, row in matched_events.iterrows():
        earthquake_info = get_event_info(row)
        image_filename = f"annotation_{earthquake_info['event_id']}.png"  # Use filename instead of path

        statistics_text = f"""
        <div class="event">
            <img src="{image_filename}" alt="Earthquake Event {earthquake_info['event_id']}">  <!-- Use filename here -->
            <div class="statistics">
                <p>Event ID: {earthquake_info['full_id']}</p>
                <p>Earthquake Time: {earthquake_info['time']}</p>
                <p>Location: Lat {earthquake_info['lat']}, Long {earthquake_info['long']}</p>
                <p>Magnitude: {earthquake_info['mag']} {earthquake_info['mag_type']}</p>
                <p>Depth: {earthquake_info['depth']} m</p>
                <p>Distance to Station: {earthquake_info['epi_distance']:.2f} km</p>
                <p>Prediction confidence: P: {earthquake_info.get('P_peak_confidence', 'N/A')}   S: {earthquake_info.get('S_peak_confidence', 'N/A')}</p>
                <p>Predicted P time: {earthquake_info.get('predicted_p_time', 'N/A')}</p>
                <p>Predicted S time: {earthquake_info.get('predicted_s_time', 'N/A')}</p>
                <p>Detected P time: {earthquake_info.get('detected_p_time', 'N/A')}</p>
                <p>Detected S time: {earthquake_info.get('detected_s_time', 'N/A')}</p>
            </div>
        </div>
        """
        html_content += statistics_text

    html_content += """
    </body>
    </html>
    """

    with open(output_html_file, 'w') as file:
        file.write(html_content)
    print(f"HTML report generated: {output_html_file}")


In [14]:
create_earthquake_report_html(matched_events, content_path, date, station, provider)

HTML report generated: D:\Dissertation\data\AM.R50D6\2024-04-23\generated\2024-04-23_report.html
